In [1]:
!python train.py

Dataset already downloaded!
Dataset already extracted!
Dateset statistics before filtering:
Number of users: 1083, with min = 100, max = 2693, and avg: 209.76731301939057
Number of venues: 38333, with min = 1, max = 1145, and avg: 5.926434142905591
Number of venue categories: 400
Dateset statistics after filtering:
Number of users: 1081, with min = 20, max = 2511, and avg: 136.6096207215541
Number of venues: 5128, with min = 10, max = 1145, and avg: 28.797776911076443
Number of venue categories: 320
Spatial graph sparsity: 0.9934740076810561
Temporal graph sparsity: 0.9182665190772997
/home/mosix11/Projects/DL/dlpenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 -

In [1]:
from src.dataset import FoursquareNYC
from src.models import TrajLSTM

ds = FoursquareNYC(num_workers=1)
ds.setup('fit')
ds.setup('test')
# model = TrajLSTM(num_user=ds.STATS['num_user'],
#                  num_pois=ds.STATS['num_pois'])

Dataset already downloaded!
Dataset already extracted!
Dateset statistics before filtering:
Number of users: 1083, with min = 100, max = 2693, and avg: 209.76731301939057
Number of venues: 38333, with min = 1, max = 1145, and avg: 5.926434142905591
Number of venue categories: 400
Dateset statistics after filtering:
Number of users: 1081, with min = 20, max = 2511, and avg: 136.6096207215541
Number of venues: 5128, with min = 10, max = 1145, and avg: 28.797776911076443
Number of venue categories: 320
Spatial graph sparsity: 0.9934740076810561
Temporal graph sparsity: 0.9182665190772997


In [2]:
ds.STATS

{'num_user': 1081,
 'num_pois': 5128,
 'num_poi_cat': 320,
 'num_time_slots': 56,
 'num_gh_P5': 115,
 'num_gh_P6': 1175,
 'num_gh_P7': 3548}

In [3]:
ds.user_train_trajectories.iloc[0]

User ID                                                              1
Venue ID             [2, 15, 12, 13, 23, 16, 14, 20, 7, 7, 13, 25, ...
Venue Category ID    [2, 14, 12, 10, 20, 15, 13, 17, 7, 7, 10, 6, 1...
Geohash P5 ID        [1, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 4, 3, 5, 6, ...
Geohash P6 ID        [1, 2, 3, 3, 4, 5, 6, 7, 7, 7, 3, 8, 9, 10, 11...
Geohash P7 ID        [1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 4, 10, 11, 12, ...
Local Time           [2012-04-07 13:42:24, 2012-04-08 14:20:29, 201...
Time Slot            [45, 53, 5, 29, 40, 41, 45, 47, 47, 47, 30, 31...
Unix Timestamp       [1333806144, 1333894829, 1333974052, 133423676...
Name: 0, dtype: object

In [15]:
a, _=ds.train_dataset.__getitem__(0)
print(a[3].shape)
print(a[4].shape)
print(a[5].shape)
# dl_train = ds.train_dataloader()
# dl_test = ds.test_dataloader()
# x, y, lens = next(iter(dl_train))
# print(len(x))
# model.validation_step(batch)

tensor([1, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 4, 3, 5, 6, 6, 1, 3, 1, 2, 2, 7, 1, 6,
        6, 2, 6, 6, 8, 3, 3, 2, 2, 3, 2, 2, 1, 6, 5])
tensor([ 1,  2,  3,  3,  4,  5,  6,  7,  7,  7,  3,  8,  9, 10, 11, 12,  1,  3,
         1, 13, 14, 15,  1, 16, 17, 18, 19, 19, 20,  3,  3, 21, 22,  3,  2, 23,
         1, 12, 24])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  4, 10, 11, 12, 13, 14,  1,  4,
         1, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,  4,  4, 25, 26,  4, 27, 28,
         1, 29, 30])


In [ ]:
import ipywidgets as widgets
import numpy as np
import torch
from IPython.display import display
widget = widgets.Output()
# with widget:
#     display(ds.poi_trajectories)
# print(ds.user_train_trajectories.iloc[400])
ds.setup(stage='fit')
ds.setup(stage='test')
dl_train = ds.train_dataloader()
dl_test = ds.test_dataloader()
x, y, lens = next(iter(dl_train)) 
user_ids = x[0]
pois = x[1]
seq_len = pois.size(1)
mask = torch.arange(seq_len).expand(len(lens), seq_len) < lens.unsqueeze(1)
expanded_user_ids = user_ids.unsqueeze(1).repeat(1, pois.size(1))
print(lens)
print(expanded_user_ids.shape)
print(pois.shape)
# print(expanded_user_ids.shape)
print(expanded_user_ids * mask)
# print(pois.shape)
# print(y[1][0])
# print(y[1].reshape(-1))
# print(lens)
# print(x[1].shape)
# print(next(iter(dl_test)))
# for batch in dl_test:
#     pass

# display(widget)

# lengths1 = np.array(ds.user_train_trajectories['Time Slot'].apply(len))
# lengths2 = np.array(ds.user_train_trajectories['Venue ID'].apply(len))
# lengths3 = np.array(ds.user_train_trajectories['Geohash ID'].apply(len))

# ds.plot_distribution(lengths)
